# ReRun : Visualisez tout rapidement et efficacement !

Bienvenue dans le monde de ReRun, votre SDK incontournable pour visualiser des données multimodales qui évoluent dynamiquement avec le temps. Les ingénieurs et chercheurs dans des domaines tels que la vision par ordinateur et la robotique utilisent ReRun pour vérifier, déboguer et présenter leurs projets avec une efficacité inégalée.

![Multimodal Timeseries Data](./img/rerun-multi-1.png)

## Points Forts

- **Modèle Open-Core :** ReRun fonctionne sur un modèle open-core, garantissant que tout dans ce référentiel reste open source et gratuit. À l'avenir, ReRun introduira un produit commercial construit sur la base solide du projet de base gratuit.

- **Adapté aux individus et aux équipes :** Le projet open source répond aux besoins des développeurs individuels, tandis que le futur produit commercial abordera spécifiquement les exigences des équipes impliquées dans la construction et l'exécution de produits de vision par ordinateur et de robotique.

- **Polyvalent et multiplateforme :** ReRun est un SDK et un moteur conçu pour visualiser et interagir avec des flux de données multimodaux. Il est simple à intégrer et à démarrer, utilisable à partir de Python, Rust et C++, et construit en Rust pour une compatibilité multiplateforme et une rapidité optimale.

- **Open Source :** ReRun s'engage envers les principes open source, avec une double licence MIT et Apache 2.

## Installation

Pour libérer la puissance de ReRun, installez-le dans votre environnement JupyterLab avec une commande simple :

In [ ]:
%%capture
! pip install -U rerun-sdk

## Commencer

Pour une expérience ReRun optimale, il est recommandé d'utiliser une session de bureau Linux, Mac ou Windows au lieu de JupyterLab. Accédez à la visionneuse ReRun complète en exécutant `rr.spawn()` dans le script suivant à partir de votre émulateur de terminal :

```python
importer la réexécution en tant que rr

rr.spawn()
```

Cependant, si JupyterLab est votre environnement préféré, ReRun y fonctionne de manière transparente. Reportez-vous à la [documentation officielle](https://www.rerun.io/docs/howto/notebook) pour plus de détails sur l'exécution de ReRun dans JupyterLab. L'exemple JupyterLab le plus simple consiste à ouvrir ReRun à l'aide des méthodes `memory_recording()` et `show()` :

In [ ]:
import rerun as rr

rec = rr.memory_recording()
rec.show(width=1024, height=768)

## Visualisation 3D : Le Cube

Découvrez la simplicité de la visualisation 3D avec ReRun en générant et en traçant des points dans l'espace :

In [ ]:
import rerun as rr
import numpy as np

rec = rr.memory_recording()

SIZE = 10

pos_grid = np.meshgrid(*[np.linspace(-10, 10, SIZE)]*3)
positions = np.vstack([d.reshape(-1) for d in pos_grid]).T

col_grid = np.meshgrid(*[np.linspace(0, 255, SIZE)]*3)
colors = np.vstack([c.reshape(-1) for c in col_grid]).astype(np.uint8).T

rr.log(
    "my_points",
    rr.Points3D(positions, colors=colors, radii=0.5)
)

rec.show(width=1024, height=768)

## Visualisation 3D avancée : L'ADN

Explorez un exemple fascinant de visualisation synthétique de données 3D en forme de double hélice :

In [ ]:
from __future__ import annotations

import argparse
from math import tau

import numpy as np

import rerun as rr  # pip install rerun-sdk
from rerun.utilities import bounce_lerp, build_color_spiral

rec = rr.memory_recording()

rr.set_time_seconds("stable_time", 0)

NUM_POINTS = 100

# points and colors are both np.array((NUM_POINTS, 3))
points1, colors1 = build_color_spiral(NUM_POINTS)
points2, colors2 = build_color_spiral(NUM_POINTS, angular_offset=tau * 0.5)
rr.log("helix/structure/left", rr.Points3D(points1, colors=colors1, radii=0.08))
rr.log("helix/structure/right", rr.Points3D(points2, colors=colors2, radii=0.08))

rr.log("helix/structure/scaffolding", rr.LineStrips3D(np.stack((points1, points2), axis=1), colors=[128, 128, 128]))

time_offsets = np.random.rand(NUM_POINTS)
for i in range(400):
    time = i * 0.01
    rr.set_time_seconds("stable_time", time)

    times = np.repeat(time, NUM_POINTS) + time_offsets
    beads = [bounce_lerp(points1[n], points2[n], times[n]) for n in range(NUM_POINTS)]
    colors = [[int(bounce_lerp(80, 230, times[n] * 2))] for n in range(NUM_POINTS)]
    rr.log(
        "helix/structure/scaffolding/beads", rr.Points3D(beads, radii=0.06, colors=np.repeat(colors, 3, axis=-1))
    )

    rr.log(
        "helix/structure",
        rr.Transform3D(rotation=rr.RotationAxisAngle(axis=[0, 0, 1], radians=time / 4.0 * tau)),
    )

rec.show(width=1024, height=768)

## Données de séries chronologiques : des tracés qui impressionnent

ReRun brille vraiment lors de la visualisation de données de séries chronologiques multimodales. L'exemple suivant démontre les prouesses de ReRun dans l'affichage de divers tracés correspondant à différentes lectures sur les mêmes données de série chronologique :

In [ ]:
from __future__ import annotations

import argparse
import random
from math import cos, sin, tau

import numpy as np
import rerun as rr  # pip install rerun-sdk

rec = rr.memory_recording()

def clamp(n, smallest, largest):  # type: ignore[no-untyped-def]
    return max(smallest, min(n, largest))


def log_bar_chart() -> None:
    rr.set_time_sequence("frame_nr", 0)
    # Log a gauss bell as a bar chart
    mean = 0
    std = 1
    variance = np.square(std)
    x = np.arange(-5, 5, 0.1)
    y = np.exp(-np.square(x - mean) / 2 * variance) / (np.sqrt(2 * np.pi * variance))
    rr.log("bar_chart", rr.BarChart(y))


def log_parabola() -> None:
    # Log a parabola as a time series
    for t in range(0, 1000, 10):
        rr.set_time_sequence("frame_nr", t)

        f_of_t = (t * 0.01 - 5) ** 3 + 1
        radius = clamp(abs(f_of_t) * 0.1, 0.5, 10.0)
        color = [255, 255, 0]
        if f_of_t < -10.0:
            color = [255, 0, 0]
        elif f_of_t > 10.0:
            color = [0, 255, 0]

        rr.log(
            "curves/parabola",
            rr.TimeSeriesScalar(
                f_of_t,
                label="f(t) = (0.01t - 3)³ + 1",
                radius=radius,
                color=color,
            ),
        )


def log_trig() -> None:
    # Log a time series
    for t in range(0, int(tau * 2 * 100.0)):
        rr.set_time_sequence("frame_nr", t)

        sin_of_t = sin(float(t) / 100.0)
        rr.log("trig/sin", rr.TimeSeriesScalar(sin_of_t, label="sin(0.01t)", color=[255, 0, 0]))

        cos_of_t = cos(float(t) / 100.0)
        rr.log("trig/cos", rr.TimeSeriesScalar(cos_of_t, label="cos(0.01t)", color=[0, 255, 0]))


def log_classification() -> None:
    # Log a time series
    for t in range(0, 1000, 2):
        rr.set_time_sequence("frame_nr", t)

        f_of_t = (2 * 0.01 * t) + 2
        color = [255, 255, 0]
        rr.log("classification/line", rr.TimeSeriesScalar(f_of_t, color=color, radius=3.0))

        g_of_t = f_of_t + random.uniform(-5.0, 5.0)
        if g_of_t < f_of_t - 1.5:
            color = [255, 0, 0]
        elif g_of_t > f_of_t + 1.5:
            color = [0, 255, 0]
        else:
            color = [255, 255, 255]
        radius = abs(g_of_t - f_of_t)
        rr.log("classification/samples", rr.TimeSeriesScalar(g_of_t, color=color, scattered=True, radius=radius))


log_bar_chart()
log_parabola()
log_trig()
log_classification()

rec.show(width=1024, height=768)


## Plongez plus profondément

Pour une compréhension approfondie de ReRun et de ses fonctionnalités, reportez-vous au [démarrage rapide officiel de ReRun Python](https://www.rerun.io/docs/quickstart/python/).

Désormais, armé de ReRun, embarquez pour un voyage de visualisation et d’exploration de données sans précédent. Bon codage !